In [267]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [268]:
data = pd.DataFrame({
    'cantidad': [100, 70, 50],
    'presa': ['M', 'L', 'G']
},index=['G', 'M', 'L'])
data

,cantidad,presa
G,100,M
M,70,L
L,50,G


In [269]:
solver = pywraplp.Solver.CreateSolver('CBC')

In [270]:
variables = {}

for viaje in range(1,9):
    for pez, cols in data.iterrows():
        variable = solver.IntVar(0, solver.infinity(), f'X_{pez}_{viaje}')
        variables[(pez, viaje)] = variable

# variables

In [271]:
variables

{('G', 1): X_G_1,
 ('M', 1): X_M_1,
 ('L', 1): X_L_1,
 ('G', 2): X_G_2,
 ('M', 2): X_M_2,
 ('L', 2): X_L_2,
 ('G', 3): X_G_3,
 ('M', 3): X_M_3,
 ('L', 3): X_L_3,
 ('G', 4): X_G_4,
 ('M', 4): X_M_4,
 ('L', 4): X_L_4,
 ('G', 5): X_G_5,
 ('M', 5): X_M_5,
 ('L', 5): X_L_5,
 ('G', 6): X_G_6,
 ('M', 6): X_M_6,
 ('L', 6): X_L_6,
 ('G', 7): X_G_7,
 ('M', 7): X_M_7,
 ('L', 7): X_L_7,
 ('G', 8): X_G_8,
 ('M', 8): X_M_8,
 ('L', 8): X_L_8}

In [272]:
# Máxima cantidad de peces por viaje
for viaje in range(1,9):
    ct = solver.Constraint(0, 30, f'Máxima cantidad de peces para el viaje {viaje}')
    for pez in data.index.values:
        ct.SetCoefficient(variables[(pez, viaje)], 1)

In [273]:
# Máxima cantidad de peces
for pez, cols in data.iterrows():
    ct2 = solver.Constraint(cols['cantidad'], cols['cantidad'], f'Maxima cantidad de peces totales {pez}')
    for viaje in range(1,9):
        ct2.SetCoefficient(variables[(pez, viaje)], 1)

In [274]:
for viaje in range(1,9):
    for pez, cols in data.iterrows():
        depredador = variables[pez, viaje] 
        presa = variables[cols['presa'], viaje]
        solver.Add(  depredador <= (2*presa + 1) )
#     _ = solver.Add( variables['M', viaje] <= variables['G', viaje], f'Depredador G -> M para viaje: {viaje}')
#     _ = solver.Add( variables['L', viaje] <= variables['M', viaje], f'Depredador M -> L para viaje: {viaje}')
#     _ = solver.Add( variables['G', viaje] <= variables['L', viaje], f'Depredador L -> G para viaje: {viaje}')

In [275]:
objective = solver.Objective()

for pez, cols in data.iterrows():
    for viaje in range(1,9):
        objective.SetCoefficient(variables[(pez, viaje)],1)

objective.SetMaximization()

In [276]:
print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 35
   Variables        : 24
     Binary         : 0
     Integer        : 24
     Continuous     : 0
Maximize
 Obj: +1 X_G_1 +1 X_M_1 +1 X_L_1 +1 X_G_2 +1 X_M_2 +1 X_L_2 +1 X_G_3 +1 X_M_3 +1 X_L_3 +1 X_G_4 +1 X_M_4 +1 X_L_4 +1 X_G_5 +1 X_M_5 +1 X_L_5 +1 X_G_6 +1 X_M_6 +1 X_L_6 +1 X_G_7 +1 X_M_7 +1 X_L_7 +1 X_G_8 +1 X_M_8 +1 X_L_8 
Subject to
 Máxima_cantidad_de_peces_para_el_viaje_1_rhs: +1 X_G_1 +1 X_M_1 +1 X_L_1  <= 30
 Máxima_cantidad_de_peces_para_el_viaje_1_lhs: +1 X_G_1 +1 X_M_1 +1 X_L_1  >= 0
 Máxima_cantidad_de_peces_para_el_viaje_2_rhs: +1 X_G_2 +1 X_M_2 +1 X_L_2  <= 30
 Máxima_cantidad_de_peces_para_el_viaje_2_lhs: +1 X_G_2 +1 X_M_2 +1 X_L_2  >= 0
 Máxima_cantidad_de_peces_para_el_viaje_3_rhs: +1 X_G_3 +1 X_M_3 +1 X_L_3  <= 30
 Máxima_cantidad_de_peces_para_el_viaje_3_lhs: +1 X_G_3 +1 X_M_3 +1 X_L_3  >= 0
 Máxima_cantidad_de_peces_para_el_viaje_4_rhs: +1 X_G_4 +1 X_M_4 +

In [277]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')


El problema tiene una solución óptima


In [278]:
solver.Objective().Value()

220.0

In [279]:
results = pd.DataFrame(
    columns=[i for i in range(1,9)],
    index = data.index.values,
    data=0
)

for pez, cols in data.iterrows():
    for viaje in range(1,9):
        results.loc[pez, viaje] = variables[pez, viaje].solution_value()

results


,1,2,3,4,5,6,7,8
G,4,17,17,7,17,12,17,9
M,17,9,9,3,8,12,8,4
L,9,4,4,1,4,6,5,17
